In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import pycountry
import re


In [2]:
df = pd.read_csv('data/finalDf.csv')
#Only the relevant ones
df = df[(df['relevant_high'] == 1) | (df['relevant']==1)]
print(df.shape)

df.head()

(8801, 46)


,Unnamed: 0,id,content,title,relevant,seen,random,2 - Adaptation,2 - Mitigation,2 - Other,...,4 - 4. Organisation_pred,7 - International_pred,7 - National_pred,7 - Subnational _pred,8 - Coastal_pred,8 - Rivers_pred,8 - Terrestrial_pred,8 - Human_pred,9 - Ex-ante_pred,9 - Ex-post_pred
10,10,505468.0,A large number of small- to medium-sized commu...,Community pond rehabilitation to deal with cli...,NaN,NaN,NaN,0.0,0.0,0.0,...,0.959119,0.008249,0.046516,0.123154,0.039962,0.874821,0.068860,0.103273,0.067134,0.057988
11,11,3286707.0,The purpose of this paper is to demonstrate th...,Vulnerability to global environmental changes:...,NaN,NaN,NaN,0.0,0.0,0.0,...,0.048519,0.007969,0.027088,0.054027,0.003420,0.005188,0.004376,0.022849,0.090995,0.109139
15,15,1962940.0,The issue of climate change has shed a new lig...,Migratory issues in climate adaptation policie...,NaN,NaN,NaN,0.0,0.0,0.0,...,0.003477,0.236410,0.458303,0.103715,0.007104,0.007903,0.014083,0.123407,0.261631,0.441075
19,19,1047270.0,(1) As on-the-ground projects come into existe...,Women's Empowerment and Climate Change Adaptat...,NaN,NaN,NaN,0.0,0.0,0.0,...,0.910039,0.020661,0.142046,0.283829,0.003045,0.003763,0.004649,0.016103,0.111538,0.292670
20,20,770190.0,Small rural coastal communities located in Atl...,Climatic and Environmental Changes Affecting C...,NaN,NaN,NaN,0.0,0.0,0.0,...,0.060801,0.016657,0.176772,0.501330,0.891267,0.044223,0.035439,0.026959,0.139272,0.180712


In [3]:
adjective_list = []
adjective_dict = {}
from bs4 import BeautifulSoup
import pycountry
import re

# read in the table with adjectives for each country
with open("data/country_adjectives.html","r") as f:
    soup = BeautifulSoup(f)

from geotext import GeoText

# define missing countries to be translated
missing_countries = {
    "Burma": "Myanmar",
    "Democratic Republic of the Congo": "Congo, The Democratic Republic of the",
    "East Timor": "Timor-Leste",
    "England": "United Kingdom",
    "European Union": None,
    "Ivory Coast": "Côte d'Ivoire",
    "Laos": "Lao",
    "Macau": "Macao",
    "Pitcairn Islands": "Pitcairn",
    "Sahrawi": "Western Sahara",
    "Scotland": "United Kingdom", 
    "Somaliland": "Somalia",
    "South Ossetia": None,
    "United States Virgin Islands": "Virgin Islands, U.S.",
    "North Korea": "Korea, Democratic People's Republic of",
    "South Korea": "Korea, Republic of",
    "Zanzibar": "Tanzania"
}
    
# Parse the table
table = soup.find("table")
for row in table.findAll("tr"):
    cols = row.findAll("td")
    if len(cols) > 0:
        cname = cols[0].find("a").getText()
        try:
            c = pycountry.countries.search_fuzzy(cname)[0]
        except:
            c_alt = missing_countries[cname]
            if c_alt is not None:
                c = pycountry.countries.search_fuzzy(c_alt)[0]
            else:
                continue
        if len(cols[1].findAll("li"))>0:
            adjectives = [x.getText() for x in cols[1].findAll("li")]
        else:
            adjectives = [cols[1].getText()]
        adjectives = [re.sub("\[.*\]","",x) for x in adjectives]
        for a in adjectives:
            adjective_list.append(a)
            adjective_dict[a] = c
        
        
        
    
{k: adjective_dict[k] for k in list(adjective_dict)[:5]}

/home/galm/software/django/tmv/venv/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


{'Abkhaz': Country(alpha_2='GE', alpha_3='GEO', name='Georgia', numeric='268'),
 'Abkhazian': Country(alpha_2='GE', alpha_3='GEO', name='Georgia', numeric='268'),
 'Afghan': Country(alpha_2='AF', alpha_3='AFG', name='Afghanistan', numeric='004', official_name='Islamic Republic of Afghanistan'),
 'Albanian': Country(alpha_2='AL', alpha_3='ALB', name='Albania', numeric='008', official_name='Republic of Albania'),
 'Åland Island': Country(alpha_2='AX', alpha_3='ALA', name='Åland Islands', numeric='248')}

In [7]:
places = []

import re

# Go through the rows of the dataframe
for i, row in df.iterrows():
    
    # Get the text we want to geoparse, join title and abstract, get rid of copyright stuff
    t = str(row['title']) + " " + str(row['content'])
    t = t.split("Copyright (C)")[0] 
    t = re.split("\([C-c]\) [1-2][0-9]{3} Elsevier",t)[0] 
    t = t.split("Published by Elsevier")[0] 
    t = t.split("Copyright. (C)")[0] 
    t = re.split("\. \(C\) [1-2][0-9]{3} ",t)[0] 
    t = re.split("\. \\xA9 [1-2][0-9]{3}", t)[0] #Copyright symbol
    t = re.split("\. \(C\) Copyright",t)[0] 
    
    for adj in adjective_list:
        if adj in t:
            places.append({
                "doc_id":row["id"],"country_code3": adjective_dict[adj].alpha_3,
                #"lat": adjective_dict[adj].lat, "lon": adjective_dict[adj].lon,
                "feature_class": "A", "feature_code": "PCLI", "word": adj,
                "country_conf": 0.95
            })
            
place_df = pd.DataFrame.from_dict(places)
print(place_df.shape)
print(place_df.doc_id.unique().shape)


place_df.head()

(3699, 6)
(3186,)


,doc_id,country_code3,feature_class,feature_code,word,country_conf
0,747442.0,GBR,A,PCLI,United Kingdom,0.95
1,747442.0,GBR,A,PCLI,UK,0.95
2,4911523.0,SVK,A,PCLI,Slovak,0.95
3,536930.0,DEU,A,PCLI,German,0.95
4,2115525.0,GBR,A,PCLI,UK,0.95


In [6]:
adjective_dict['Indian']


Country(alpha_2='IN', alpha_3='IND', name='India', numeric='356', official_name='Republic of India')

In [23]:
place_df.to_csv('data/geos_adjectives.csv', encoding = 'utf-8', index=False)